***Wolves vs Prey***

This code is intended to produce a simulation of wolf and prey populations over a specified time series.  The classroom discussion led to the following PRELIMINARY details about the simulation:

Here is [an interesting reference](https://www.researchgate.net/publication/239927489_Kill_Rates_and_Predation_by_Wolves_on_Ungulate_Populations_in_Bialowieza_Primeval_Forest_Poland?enrichId=rgreq-644c6d606fbbf71d082d4cbb0ade1da6-XXX&enrichSource=Y292ZXJQYWdlOzIzOTkyNzQ4OTtBUzo5ODg2MTU3NDAwMDY1M0AxNDAwNTgxODQ2ODMz&el=1_x_2&_esc=publicationCoverPdf) for wolf-prey dynamics.

**Goal:** Predict wolf and prey population N years beyond initial conditions

**Global assumptions:**
  * Isolated habitat with no human interactions
  * Restricted habitat area, forcing variable density
  * Modeling averages over single year periods (time step = 1 year)
  * No animal migration

  
**Input:**
  * Initial wolf population, assume equal male/female?  Ages?  Numbers of each.
  * Prey population, within limited species (rabbits, deer, moose, ??).
  * Animal reproduction rate: 2->N average per year (2->N is a bit arbitrary).  Does this depend on density?
  * Wolf predation rate per species (depends on population density?)
  * Habitat size (determines densities)
  * Weather adversity index (separately for wolves and prey)
  * Disease advertity index (separately for wolves and prey)
  * Average animal lifetimes

  
**Model details/assumptions:**
  * Require relationships between preditor and prey densities vs predation rate
  * Require relationship between prey density and food competition?
      * And also weather adversity?
      * Or assume general abundance?
  * No dependence of predation rate on age?
  * Require relationship between weather adversity and animal death rate
  * Require relationship between disease adversity and animal death rate
  * Require list of animal attributes & their population distribution
  * Require means to pass down traits to offspring (eg, hunting skill or reproduction rate)
  * Require model for prey density vs preditor survival rate



**Model implementation:**
  * Define class for a given animal,
      * Decorate with required variables to track animal attributes
  * Generate arrays of each species type, populated according to initial conditions
  * Iterate one year time steps
      * Calculate animal densities
      * Determine death events, remove dead animals from arrays (dead animals don't reproduce)
      * Determine birth events, adding new animals with new attributes drawn from parents
      * Repeat until desired future year

In [1]:
###################################
## Global simulation parameters:
## nWolf_init: Initial wolf population size (integer)
## nPrey_init: Initial prey population size (integer)
## wolf_age_init: Average age of initial wolf population
## prey_age_init: Average age of initial prey population
##
## wolf_repro_rate: Average number of wolf pups per year from an average wolf (integer)
## prey_repro_rate: Average number of prey offspring per year from an average animal (integer)
##
## wolf_pred_rate: Average number of prey animals each wolf consumes per year (integer)
## wolf_pred_min: Minimum number of prey animals that a wolf must consume per year to survive (integer)
##
## wolf_lifetime: Average wolf natural lifetime in years (integer)
## prey_lifetime: Average prey natural lifetime in years (integer)
##
## habitat_size: Size of habitat in square kilometers 
## weather_idx: Weather & disaster adversity index [0,1]
## disease_idx: Disease adversity index [0,1]
##
## wolf_pop_std: Standard deviation of wolf population 
###################################


## Initial population variables
nWolf_init = 10
nPrey_init = 1000
wolf_age_init = 5
prey_age_init = 2

## Reproduction rate & lifetime variables
wolf_repro_rate = 2  # Average wolf pup rate (average is 4, but we divide by 2 because we don't track gender)
prey_repro_rate = 3  # Average prey offspring rate (again, division by 2 baked in)
wolf_repro_rate_std = 0.25 # 25% standard deviation on wolf 
prey_repro_rate_std = 0.25 # 25% standard deviation

wolf_lifetime = 15 # This is the theoretical maximum
prey_lifetime = 10 # This is the theoretical maximum

## Wolf predation variables
wolf_pred_rate = 40 # number of prey each wolf consumes each year on average
wolf_pred_min = 20 # minimum number of prey a wolf must consume per year to avoid dying of hunger
wolf_pred_std = 0.25 # standard deviation of wolf prey consumption, 20%

# environment variables
habitat_size = 500 # square kilometers
weather_idx = 0.5 
disease_idx = 0.5

In [2]:
# This is the class that we'll use for an individual wolf.  It can be edited as needed.
class wolf:
    def __init__(self,age,pred_idx,repro_idx):
        self.age = age #This is the age of the wolf, which will increment each year
        self.pred_idx = pred_idx #This is an index [0,1] to describe this wolf's predatory skill
        self.repro_idx = repro_idx #This is an index [0,1] to describe this wolf's reproduction success rate
    
    def increaseAge(self): #increment age by one year
        self.age = self.age + 1
        
    def print(self): # print the characteristics of an individual wolf for debugging purposes
        print(f"{self.age}\t{self.pred_idx}\t{self.repro_idx}")
        
# This is the class that we'll use for an individual prey animal.  It can be edited as needed.
class prey:
    def __init__(self,age,repro_idx):
        self.age = age #This is the age of the prey animal, which will increment each year
        self.repro_idx = repro_idx #This is an index [0,1] to describe this animal's reproduction success rate
    
    def increaseAge(self): #increment age by one year
        self.age = self.age + 1
        

In [3]:
# These functions will create the initial populations of 
def initializeWolves(nWolves, avgAge):
    # create initial set of randomized wolves: newWolf = wolf(randomAge, randomPidx, randomRidx)
    # return array of wolf objects
    # set all characteristics to 0.50 randomized by the standard deviation variable above
    # should this 0.50 be default for the simulation or driven by a configurable variable?
    
    # adding one wolf by hand as an example
    wolfpack = [wolf(1,0.50,0.50)]
    return wolfpack

def initializePrey(nPrey, avgAge):
    # create initial set of randomized prey: newPrey = prey(randomAge,randomPidx,randomRidx)
    # return array of prey objects
    # set reproduction rate index to 0.50 randomized by the standard deviation variable above
    # should this 0.50 be default for the simulation or driven by a configurable variable?
    
    # just adding two prey by hand as an example
    preyherd = [prey(2,0.50),prey(3,0.50)]
    return preyherd


In [16]:
# These functions increase and decrease the wolf and prey populations

def deathEvent(wolves,prey):
    # First simulate how many prey each wolf consumes, remove wolves that didn't consume enough prey
    # Next randomly remove prey that were eaten by wolves
    # The weather adversity index should play a role in the efficiency of wolves catching prey.
    # But does the adversity have a greater impace on the wolves or the prey?
    #
    # Death events should also be randomly applied to all animals in addition to the prey consumed by wolves
    # As an animal reaches its maximim lifetime, the chances of randomly dying should increase.
    # The random chance should include the weather and disease adversity indices.
    # Should the disease adversity index depend on population density?
    # Should the weather and disease adversity indices impact wolves and prey equally?
    return

def birthEvent(wolves,prey):
    # This function should come after the death event, modeling that only healthy and surviving creatures reproduce
    # Each individual should have the opportunity to give rise to offspring at a randomized rate.
    # The weather adversity index may play a role, perhaps differently for each species of animal.
    # New animals should be added to the appropriate arrays with an age of 0.
    # New animals should be initialized with characteristics that are randomly drawn from the parent's characteristics
    import numpy as np
    from random import gauss
    # First, let's take care of wolf birth event
    avgPups = int(len(wolves)/2) * wolf_repro_rate
    stdPups = int(len(wolves)/2) * wolf_repro_rate_std
    
    # Question: there are both a global wolf repro rate and an invidual wolf repro idx; not sure which to use or both, and how those factor into the birth events
    # so I'm taking my best guess - MJB
    nPups = 0 # accumulator for number of pups born
    w = 0 # idx for traversing wolf list
    np.random.shuffle(wolves) # basing birth pairs in order, so shuffle the list so that wolves in the list are not always next to each other
    while w + 1 < len(wolves):
        # get an average probability for reproduction between a pair of wolves
        birth_prob = (wolves[w].repro_idx + wolves[w+1].repro_idx)/2
        # add pups if repro succeeds
        if (np.random.uniform(0,1,1) <= birth_prob):
            nPups += int(round(gauss(avgPups, stdPups)))
        w += 2
    # now add wolf pups to the wolf list
    for i in range(nPups):
        age = 0 # newborn; age is 0
        # assume that the skill and repro rate distributions for pups are uniform
        # A normal distribution likely makes more sense here, but need to consider what the stdevs should be, then
        skill = np.random.uniform(0,1)
        repro = np.random.uniform(0,1)
        wolves.append(wolf(age,skill,repro))

    # Then we can do a similar process for the prey birth event
    # TODO: add prey birth event process
    return

def incrementAges(wolves,prey):
    # This function should just increase the age of all existing animals
    for instance in wolves:
        instance.increaseAge()
        
    for instance in prey:
        instance.increaseAge()
        
    return

In [18]:
def _testBirthEvent():
    import numpy as np
    wolves = []
    for i in range(5):
        wolves.append(wolf(1,0.5,0.5))
    #for i in range(4):
    #    temp = wolf(1,0.5,0.5)
    #    wolves.append(temp)
    print("Before birth event: wolf pack")
    for w in range(len(wolves)):
        wolves[w].print()
    birthEvent(wolves,[])
    print("After birth event: wolf pack")
    for w in range(len(wolves)):
        wolves[w].print()
_testBirthEvent() 


Before birth event: wolf pack
1	0.5	0.5
1	0.5	0.5
1	0.5	0.5
1	0.5	0.5
1	0.5	0.5
After birth event: wolf pack
1	0.5	0.5
1	0.5	0.5
1	0.5	0.5
1	0.5	0.5
1	0.5	0.5
0	0.9797514972596956	0.9132427174256483
0	0.6136563623308189	0.8671199941276889
0	0.31863634538958163	0.45383068661196246
0	0.25370926753424694	0.6306186932778485
0	0.13202014686116137	0.57673482173678


In [25]:
def generateExperiment(years):
    # create our initial animal arrays
    _wolves = initializeWolves(nWolf_init, wolf_age_init)
    _prey = initializePrey(nPrey_init, prey_age_init)
    
    #now iterate on the animals
    for i in range(years):
        print("Simulating year ",i)
        incrementAges(_wolves,_prey) # increment ages first so that surviving animals age and new animals have age=0
        deathEvent(_wolves,_prey) # some animals die
        birthEvent(_wolves,_prey) # new animals born
    
    return _wolves, _prey

In [26]:
wolfpack, preyherd = generateExperiment(5)

Simulating year  0
Simulating year  1
Simulating year  2
Simulating year  3
Simulating year  4


In [27]:
print("Wolf population: ",len(wolfpack))
print("Prey population: ",len(preyherd))

print("Wolf 0 age: ",wolfpack[0].age)
print("Prey 0 age: ",preyherd[0].age)

Wolf population:  1
Prey population:  2
Wolf 0 age:  6
Prey 0 age:  7
